In [1]:
#Example Text
simple_text=["Good Job Done","Very Good Job Done","Bad Job Done","Very Bad Job Done","Bad Job"]

In [2]:
from sklearn.feature_extraction.text import CountVectorizer
vect=CountVectorizer()

#Fit : Learn the "Dictionary" of the data provided. for example: simple-text here
vect.fit(simple_text)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [3]:
# to see the dictionary made from data
vect.get_feature_names()

['bad', 'done', 'good', 'job', 'very']

In [4]:
data_matrix=vect.transform(simple_text)
print(data_matrix)

  (0, 1)	1
  (0, 2)	1
  (0, 3)	1
  (1, 1)	1
  (1, 2)	1
  (1, 3)	1
  (1, 4)	1
  (2, 0)	1
  (2, 1)	1
  (2, 3)	1
  (3, 0)	1
  (3, 1)	1
  (3, 3)	1
  (3, 4)	1
  (4, 0)	1
  (4, 3)	1


In [5]:
dense_matrix=data_matrix.toarray()
print(dense_matrix)

[[0 1 1 1 0]
 [0 1 1 1 1]
 [1 1 0 1 0]
 [1 1 0 1 1]
 [1 0 0 1 0]]


In [6]:
import pandas as pd
df=pd.DataFrame(data_matrix.toarray(),columns=vect.get_feature_names())
print(df)

   bad  done  good  job  very
0    0     1     1    1     0
1    0     1     1    1     1
2    1     1     0    1     0
3    1     1     0    1     1
4    1     0     0    1     0


In [7]:
#Reading sms.txt file
sms=pd.read_csv("sms.txt",header=None,names=['label','message'],sep='\t')

In [8]:
sms.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [9]:
sms.label.value_counts()

ham     4825
spam     747
Name: label, dtype: int64

In [10]:
sms.shape

(5572, 2)

In [11]:
X=sms.message
y=sms.label

In [13]:
#Spilt X and y in training and testing data sets
#when test_size is not given, its default value is 0.25

from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,y)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(4179,)
(1393,)
(4179,)
(1393,)


In [14]:
vect=CountVectorizer()
vect.fit(X_train) #making dictionary
X_train_matrix=vect.transform(X_train)

In [15]:
X_train_matrix

<4179x7526 sparse matrix of type '<class 'numpy.int64'>'
	with 55492 stored elements in Compressed Sparse Row format>

In [ ]:
#7526 unique words exist in X_train messages. these are called dictionary for these messages.

In [17]:
X_test_matrix=vect.transform(X_test)
X_test_matrix

<1393x7526 sparse matrix of type '<class 'numpy.int64'>'
	with 17392 stored elements in Compressed Sparse Row format>

In [18]:
from sklearn.naive_bayes import MultinomialNB

nb=MultinomialNB()
nb.fit(X_train_matrix,y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [19]:
#nb is the ML model which you have made
y_predict=nb.predict(X_test_matrix)

In [20]:
print(y_predict)

['spam' 'ham' 'ham' ... 'ham' 'ham' 'ham']


In [22]:
print(nb.score(X_test_matrix,y_test))

0.9921033740129217


In [23]:
from sklearn import metrics
metrics.accuracy_score(y_test,y_predict)

0.9921033740129217

In [24]:
metrics.confusion_matrix(y_test,y_predict)

array([[1210,    2],
       [   9,  172]], dtype=int64)

In [25]:
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression()
lr.fit(X_train_matrix,y_train)
print(lr.score(X_test_matrix,y_test))

0.9842067480258435


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [26]:
# print message text for the false positives (FP)
# ham incorrectly classified as spam

X_test[(y_predict=="spam") & (y_test=="ham")]

5046    We have sent JD for Customer Service cum Accou...
4600              Have you laid your airtel line to rest?
Name: message, dtype: object

In [27]:
# print message text for the false negatives (FN)
# spam incorrectly classified as ham

X_test[(y_predict=="ham") & (y_test=="spam")]

68      Did you hear about the new "Divorce Barbie"? I...
869     Hello. We need some posh birds and chaps to us...
943     How about getting in touch with folks waiting ...
788     Ever thought about living a good life with a p...
4949    Hi this is Amy, we will be sending you a free ...
4298    thesmszone.com lets you send free anonymous an...
1269    Can U get 2 phone NOW? I wanna chat 2 set up m...
3302             RCT' THNQ Adrian for U text. Rgds Vatian
3742                                        2/2 146tf150p
Name: message, dtype: object

In [28]:
X_test[4949]

'Hi this is Amy, we will be sending you a free phone number in a couple of days, which will give you an access to all the adult parties...'